In [17]:
import os
import timm
import torch
from torch.utils.data import DataLoader
import numpy as np
import wandb
wandb.init(project="adversarial-paper-ift6164", entity="j-bytes")
import torchattacks
import torchvision.transforms as transforms
from torchvision import datasets
import albumentations as A
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
from utils import training_loop,validation_loop,randAugment
from tqdm.notebook import tqdm
from torch.utils.data import Dataset
device= "cuda" if torch.cuda.is_available() else "cpu"
names = ["airplane","automobile","bird","cat","deer","dog","frog","horse","ship","truck"]
num_classes=len(names)


[notice] A new release of pip available: 22.2.1 -> 22.3.1
[notice] To update, run: pip install --upgrade pip


In [18]:
import torchattacks
mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)

model1 = timm.create_model("resnext50_32x4d",pretrained=True,num_classes=num_classes).to(device) #trained on imagenet
model2 = timm.create_model("convnext_small",pretrained=True,num_classes=num_classes).to(device) # will be trained on cifar10

#ops = randAugment(N=2,M=5,p=0.5,mode="all",cut_out=True)
train_transform=transforms.Compose(
    [
        #transforms.RandAugment(num_ops=5,magnitude=2),

        transforms.RandomHorizontalFlip(),
        transforms.RandomCrop(32, padding=4),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
        transforms.ToTensor(),
        transforms.Normalize(mean,std)
    ]
)

val_transform=transforms.Compose(
    [
                transforms.Resize(32),
                transforms.CenterCrop(32),
                transforms.ToTensor(),
                transforms.Normalize(mean, std),
    ]
)

def target_transform(label) :
     label_vector = torch.zeros((num_classes))
     label_vector[label]=1
     return label_vector

# class Cifar10SearchDataset(datasets.CIFAR10):
#     def __init__(self, root="~/data/cifar10", train=True, download=True, transform=None):
#         super().__init__(root=root, train=train, download=download, transform=transform)
#
#     def __getitem__(self, index):
#         image, label = self.data[index], self.targets[index]
#
#         if self.transform is not None:
#             transformed = self.transform(image=image)
#             image = transformed["image"]
#
#         return image, label
training_data = datasets.CIFAR10(
    root="/mnt/f/data",
    train=True,
    download=True,
    transform=train_transform,
    #target_transform=target_transform

)

test_data = datasets.CIFAR10(
    root="/mnt/f/data",
    train=False,
    download=True,
    transform=val_transform,
    #target_transform=target_transform

)





In [19]:
train_loader = DataLoader(
    training_data,
    num_workers=8,
    batch_size=256,
    shuffle=True
)
test_loader= DataLoader(
    test_data,
    num_workers=8,
    batch_size=256
)

Files already downloaded and verified
Files already downloaded and verified


In [20]:

metrics = {
    "accuracy" : [],
    "train_loss" : [],
    "val_loss" : [],
}
keep_training=True
criterion = lambda  pred, true : torch.nn.functional.cross_entropy(torch.softmax(pred,dim=1),true)
for model in  [model2,model1] :
    if os.path.exists(f"{model.default_cfg['architecture']}.pt") :
        model.load_state_dict(torch.load(f"{model.default_cfg['architecture']}.pt"))
        model = model.to(device)
        val_loss,results = validation_loop(model,test_loader,criterion,device,autocast=False)
        accuracy = (results[0].cpu().numpy()== np.argmax(torch.softmax(results[1].cpu(),dim=1).numpy(),axis=1)).mean()
        print(f"Model : {model.default_cfg['architecture']} , Accuracy : {accuracy}")
    else :
        accuracy=0
    if keep_training and accuracy<0.85 :
        optimizer = torch.optim.AdamW(model.parameters(),lr=1e-4,weight_decay=1e-3)


        scaler = torch.cuda.amp.GradScaler()
        max_epoch = 30
        scheduler=torch.optim.lr_scheduler.OneCycleLR(max_lr=1e-4,optimizer=optimizer,epochs=max_epoch,steps_per_epoch=len(train_loader))

        for epoch in (pbar:=tqdm(range(0,max_epoch),position=0,leave=True)) :
            loss = training_loop(model,tqdm(train_loader,position=1,leave=False),optimizer=optimizer,criterion=criterion,device=device,scaler=scaler,clip_norm=0,autocast=False,scheduler=scheduler,epoch=max_epoch)

            val_loss,results = validation_loop(model,test_loader,criterion,device,autocast=True)
            accuracy = (results[0].cpu().numpy()== np.argmax(torch.softmax(results[1].cpu(),dim=1).numpy(),axis=1)).mean()
            pbar.set_description(f"Training loss : {loss/len(train_loader)} ,  validation loss : {val_loss/len(test_loader)} , accuracy : {accuracy}")
            metrics["accuracy"].append(accuracy)
            metrics["train_loss"].append(loss/len(train_loader))
            metrics["val_loss"].append(val_loss/len(test_loader))


        torch.save(model.state_dict(), f"{model.default_cfg['architecture']}.pt")
        model.to("cpu")



In [21]:

# If, images are normalized:
# atk.set_normalization_used(mean=[...], std=[...])


model1 = model1.to(device)
model2 = model2.to(device)
#train_loader.batch_size=128
optimizer = torch.optim.AdamW(model1.parameters(),lr=1e-4)
scheduler=torch.optim.lr_scheduler.OneCycleLR(optimizer,max_lr=1e-2,epochs=50,steps_per_epoch=len(test_loader))

#atk.loss = torch.nn.MSELoss()
mse = torch.nn.MSELoss()
ce = torch.nn.CrossEntropyLoss()
eps=0#0.01/256
model1.train()
model2.eval()
wandb.config({
    "eps" : eps,
    "model1" : model1.default_cfg["architecture"],
    "model2" : model2.default_cfg["architecture"],

})
wandb.watch(model1)
wandb.watch(model2)
for epoch in (pbar:=tqdm(range(0,50),leave=True,position=0)) :
    running_loss=0
    
    #atk.loss = lambda pred,true : mse(torch.nn.functional.softmax(pred,dim=1).float(),torch.nn.functional.one_hot(true,10).float())
    #atk.loss = lambda pred,true : ce(torch.nn.functional.softmax(pred,dim=1),true)
    std_logits=None
    sensitivity = torch.zeros((10,))
    ex=0

    accuracy1 = []
    accuracy2 = []
    for images,labels in tqdm(test_loader,position=1,leave=False) :
        atk = torchattacks.PGD(model1, eps=eps, alpha=eps/3, steps=100)
        #atk.loss = mse
        images,labels = images.to(device,non_blocking=True),labels.to(device,non_blocking=True)





        with torch.cuda.amp.autocast(enabled=True) :
          with torch.no_grad() :
              logit_model2_img        = model2(images).detach()
                
          adv_images = atk(images,labels)# logit_model2_img) # supposed to use labels! ->sensitibity 1.06 with eps=.01/256
          adv_images = adv_images.to(device,non_blocking=True)

          #logit_model1_img        = model1(images)
          logit_model1_adv_img    = model1(adv_images)
          with torch.no_grad() :
              #logit_model2_img        = model2(images).detach()
              logit_model2_adv_img    = model2(adv_images).detach()




        loss = mse(logit_model2_adv_img,logit_model1_adv_img)#+mse(logit_model2_img,logit_model1_img)
        #loss = ((logit_model2_adv_img-logit_model1_adv_img)**2).mean()

        loss.backward()
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

        running_loss+=loss.item()


        if std_logits is None :
            std_logits = logit_model2_img.to("cpu",non_blocking=True)
        else :
            std_logits = torch.cat((std_logits,logit_model2_img.to("cpu",non_blocking=True)),dim=0)


        sensitivity += torch.mean(torch.abs(logit_model2_img-logit_model2_adv_img),dim=0).to("cpu",non_blocking=True)
        baseline  = (torch.argmax(torch.softmax(logit_model2_img,dim=1),dim=1) == labels).float().mean().round(decimals=2)
        accuracy1.append( (torch.argmax(torch.softmax(logit_model1_adv_img,dim=1),dim=1) == labels).float().mean().round(decimals=2) )
        accuracy2.append( (torch.argmax(torch.softmax(logit_model2_adv_img,dim=1),dim=1) == labels).float().mean().round(decimals=2) )

    pbar.set_description(f"Sensitivity : {torch.mean(sensitivity/len(test_loader)/torch.abs(std_logits)).round(decimals=2)} ,  loss : {round(loss.item(),ndigits=2)} , acc1 : {accuracy1} , acc2 : {accuracy2} , baseline : {baseline}")

    metrics = {
        "sensitivity"   : torch.mean(sensitivity/len(test_loader)/torch.abs(std_logits)).round(decimals=2),
        "loss"          : running_loss/len(test_loader),
        "accuracy1"     : np.mean(accuracy1),
        "accuracy2"     : np.mean(accuracy2)
    }
    wandb.log(metrics,epoch=epoch)


        #adv_pred = torch.softmax(model1(adv_images),dim=1)




Model : convnext_small , Accuracy : 0.9156


wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
ERROR:root:dropped chunk 404 Client Error: Not Found for url: https://api.wandb.ai/files/j-bytes/adversarial-paper-ift6164/379k44go/file_stream
NoneType: None


Model : resnext50_32x4d , Accuracy : 0.8462


  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/196 [00:00<?, ?it/s]

...

wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
ERROR:root:dropped chunk 404 Client Error: Not Found for url: https://api.wandb.ai/files/j-bytes/adversarial-paper-ift6164/379k44go/file_stream
NoneType: None


  0%|          | 0/196 [00:00<?, ?it/s]

  0%|          | 0/196 [00:00<?, ?it/s]

  0%|          | 0/196 [00:00<?, ?it/s]

wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
ERROR:root:dropped chunk 404 Client Error: Not Found for url: https://api.wandb.ai/files/j-bytes/adversarial-paper-ift6164/379k44go/file_stream
NoneType: None


  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

...

  0%|          | 0/40 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

KeyboardInterrupt: 